In [ ]:
import requests
import pandas as pd
from IPython.display import clear_output, display
from bs4 import BeautifulSoup
import re

In [ ]:
BASE_URL = "https://www.timingproductions.com/results-site/houston"

URL = "https://e.timingproductions.com/dp/26aa600036849de1dcdc4f2b886a?rnd=1716250877204"


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
options = Options()
options.headless = True
manager = ChromeDriverManager()
driver = webdriver.Chrome(service=Service(manager.install()), options=options)

In [ ]:
driver.get(BASE_URL)

In [ ]:
el=driver.find_element(By.NAME, "Value2_1")
el.click()
#el.send_keys(Keys.DOWN) # previous year
el.send_keys(Keys.ENTER)

In [ ]:
el=driver.find_element(By.NAME, "Value3_1")
el.click()
el.send_keys('M')
el.send_keys(Keys.ENTER)

In [ ]:
el=driver.find_element(By.NAME, "Value4_1")
el.click()
el.send_keys('R')
el.send_keys(Keys.ENTER)

In [ ]:
el=driver.find_element(By.NAME, "searchID")
el.click()

In [ ]:

el=Wait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR,f"input.cbResultSetJumpToTextField")))

total_pg = int(re.match(r'.*\sof\s(\d+)\s*',el.get_attribute('title'))[1])
total_pg

In [ ]:
df_final = None

for page_nbr in range(1,total_pg+1):

    Wait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR,f"input.cbResultSetJumpToTextField[value='{page_nbr}'")))

    clear_output()
    display(page_nbr)

    pg= BeautifulSoup(driver.page_source)
    rows=pg.find_all('tr', class_='cbResultSetDataRow')
    df=pd.DataFrame([sum([c.text.replace(u'\xa0',' ').replace('\n','|').strip().split('|') for c in r.find_all('td')], []) for r in rows])

    if df_final is None:
        df_final=df
    else:
        df_final=pd.concat([df_final,df])

    try:
        btn=driver.find_element(By.CSS_SELECTOR, "a[data-cb-name='JumpToNext']" )
        btn.click()
    except NoSuchElementException:
        break

    
    



In [ ]:
df_final.reset_index(inplace=True, drop=True)
df_final.columns=[ 'Name', 'sex-age', 'bib', 'race', 'origin', 'status', 'place', 'place_gender', 'place_group', 'time', 'min_mile', 'void1', 'void2']
df_final.head(10)

In [ ]:
df_final.status.value_counts()

In [ ]:
driver.close()


In [ ]:
df_final['bib']=df_final.bib.str.replace(r'\s*Bib\s*','', regex=True)

In [ ]:
df_final[['sex','age']] =df_final['sex-age'].str.split('-', expand=True)
df_final['age']=df_final['age'].str.strip()

In [ ]:
df_final.drop(['race','void1', 'void2', 'sex-age'], inplace=True, axis=1)
df_final.head(10)

In [ ]:
df_final.to_csv("houston_2024_full.csv")

In [ ]:
df_final[df_final.status==''][['sex','age','time']].to_csv('houston_2024.csv')